# Expense planning tool

In [1]:
# Expenses planning tool

import numpy as np
import pandas as pd

name_list = []
expense_list = []
name_no = 1
name_loop = True        # Used in Part 1


# Part 1. Create a list of names with an option to finish and proceed.

while name_loop != False:
    name = input('Provide a name of the person or "N" to stop: ')
    if name.lower() == 'n':   
        print('next step')
        name_loop = False
    else:
        name_list.append(name)

# Part 2. Synthetically create "for in" loop using "while"
        # Loop through names difinitely 
        # Loop through expenses indifinitely. "N" to end loop
        # Create a list of expenses, who paid, how much, split between who

index = 0
while index < len(name_list):
    
    expense_loop = True
    while expense_loop != False:                   
        expense_name = input('Name an expense of ' + str(name_list[index]) + ' or "N" to finish: ')

        if expense_name.lower() == 'n':
            expense_loop = False
            print('end')
            break
                        
        else:
            expense_price = input('How much was the expense?: ')
            print(name_list)
            split_between = input("Split between who?: use index like '1,3,4' or '0' for everyone: ")
            
            if split_between == '0':
                List = [1]*len(name_list)                                 # Creates a list of [True, True, ...] if '0' for Everyone
                                                                          # Adds the expense to the expense_list
                expense_list.append([expense_name, name_list[index], int(expense_price)] + List)
                
            else:  
                List = [0]*len(name_list)                                 # Creates a list of [False, False, ...] to make a replacement later
                
                                                # converts a string "1,2,  5, 6,3" into a list. Removes spaces, convers to list with delimeter ","
                                                # converts to int in map object, then list object
                split_between_int = list(map(int, split_between.replace(' ','').split(sep = ',')))
                for X in split_between_int:
                    List[X - 1] = 1
                expense_list.append([expense_name, name_list[index], int(expense_price)] + List)    
                 
            
    index += 1   # part of the synthetical "while" loop

# Probable input data for the next part
# name_list = ['Slava', 'Bobby', 'Alex', 'Sabrine']

# expense_list = [['Cottage', 'Slava', 400, 1, 1, 1, 1],
# ['Booze', 'Slava', 100, 1, 1, 1, 0],
# ['Sausages', 'Bobby', 20, 1, 1, 1, 0],
# ['Car', 'Alex', 150, 1, 0, 1, 1]]    
    
# Part 3. Create a dataframe using the lists

        
labels = ['Expense', 'Who paid', 'How much?'] + name_list      # Create a list for columns
df = pd.DataFrame.from_records(expense_list, columns=labels)   # Create a dataframe


for label, row in df.iterrows():
    exp_each = df.loc[label,'How much?'] / int(df.loc[[label], name_list].sum(axis = 1))  # Devide expense on amount of people the expense is split between
    df.loc[label] = df.loc[label].replace(1, round(exp_each, 2))                          # Replace False or "0" with this value
    


df = df.append(df.sum(numeric_only=True), ignore_index=True)       # Create a bottom line as a SUM and name it 'Expenses Total'

df.iloc[[-1], [0]] = 'Expenses Total'


df=df.set_index('Expense')                                         # Name the index columns as "Expense"

paid_values = [0]* len(name_list) + [0,0]                          # Create a line "Paid" with 0 values. Will replace later
df.loc['Paid'] = paid_values



people_paid = []                                                   # Extract people from "expense_list" to "people_paid" list for aggregations
for x in range(len(expense_list)):
    people_paid.append(expense_list[x][1])
people_paid = list(set(people_paid))


sub_table = df.groupby(['Who paid']).sum()                         # Create an aggregation table to check how much each person from "people_paid" paid in total
for  value in people_paid:                                         # Look up a value from the table and replace the value in df.loc["Paid"]
    df.loc[['Paid'],[value]] = -(sub_table.loc[value,'How much?'])

    
df.loc['Total Balance'] = [0]* len(name_list) + [0,0]              # Create a line "Total Balance" with 0 values
for x in range(2, 2 + len(name_list)):                             # which is a sum of "Expense Total" and "Paid"
    df.iloc[-1, x ] = df.iloc[-2,x] + df.iloc[-3,x]
   


for x in range(2):                                                  # Replace bottom left values with fancy arrows
    df.iloc[-2, x] = '=>'
    df.iloc[-1, x] = '=>'

df.iloc[-3, 0] = '=>'

df

Provide a name of the person or "N" to stop: Djina
Provide a name of the person or "N" to stop: Slava
Provide a name of the person or "N" to stop: Alex
Provide a name of the person or "N" to stop: Vanja
Provide a name of the person or "N" to stop: N
next step
Name an expense of Djina or "N" to finish: Booze
How much was the expense?: 100
['Djina', 'Slava', 'Alex', 'Vanja']
Split between who?: use index like '1,3,4' or '0' for everyone: 0
Name an expense of Djina or "N" to finish: N
end
Name an expense of Slava or "N" to finish: Gas
How much was the expense?: 20
['Djina', 'Slava', 'Alex', 'Vanja']
Split between who?: use index like '1,3,4' or '0' for everyone: 2
Name an expense of Slava or "N" to finish: N
end
Name an expense of Alex or "N" to finish: condoms
How much was the expense?: 15
['Djina', 'Slava', 'Alex', 'Vanja']
Split between who?: use index like '1,3,4' or '0' for everyone: 1,2
Name an expense of Alex or "N" to finish: N
end
Name an expense of Vanja or "N" to finish: N
end


,Who paid,How much?,Djina,Slava,Alex,Vanja
Expense,,,,,,
Booze,Djina,100,25.0,25.0,25.0,25.0
Gas,Slava,20,0.0,20.0,0.0,0.0
condoms,Alex,15,7.5,7.5,0.0,0.0
Expenses Total,=>,135,32.5,52.5,25.0,25.0
Paid,=>,=>,-100.0,-20.0,-15.0,0.0
Total Balance,=>,=>,-67.5,32.5,10.0,25.0
